### UCB fitting

In [ ]:
import numpy as np
import pandas as pd
import mab_subjects
from banditpy.models import DecisionModel
from banditpy.models.policy import EmpiricalUCB, RLUCB, BayesianUCB

exps = mab_subjects.mostly_unstruc.allsess + mab_subjects.mostly_struc.allsess

ucb_df = []
for e, exp in enumerate(exps[6:7]):
    print(exp.sub_name)

    task = exp.b2a.filter_by_trials(100, 100)
    task.auto_block_window_ids()

    # mask_block1 = task.block_ids == 1
    # task_block1 = task._filtered(mask_block1)

    # mask_block2 = task.block_ids == 2
    # task_block2 = task._filtered(mask_block2)

    model = DecisionModel(task, policy=BayesianUCB())
    # model.fit(n_starts=30, progress=True, n_jobs=5)
    model.fit(
        method="de",
        de_popsize=20,
        de_maxiter=400,
        de_tol=1e-7,
        n_starts=2,
        progress=True,
        n_jobs=1,
        seed=None,
    )
    model.print_params()

    # model_block1 = DecisionModel(task_block1, policy=EmpiricalUCB())
    # model_block1.fit()

    # model_block2 = DecisionModel(task_block2, policy=EmpiricalUCB())
    # model_block2.fit()

    df = pd.DataFrame(
        {
            "name": exp.sub_name,
            "param_all": list(model.params.keys()),
            "param_values_all": list(model.params.values()),
            # "param_block1": list(model_block1.params.keys()),
            # "param_values_block1": list(model_block1.params.values()),
            # "param_block2": list(model_block2.params.keys()),
            # "param_values_block2": list(model_block2.params.values()),
            "grp": exp.group_tag,
            "dataset": exp.data_tag,
            "lesion": exp.lesion_tag,
            "mode": "classic",
        }
    )
    ucb_df.append(df)

ucb_df = pd.concat(ucb_df, ignore_index=True)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from neuropy import plotting
from mab_colors import colors_2arm
from statplotannot.plots.plot_utils import fix_legend

fig = plotting.Fig(5, 4)

df = mab_subjects.GroupData().ucb_fitting_results.latest
for s, sfx in enumerate(["_all", "_block1", "_block2"]):
    ax = fig.subplot(fig.gs[s])

    sns.barplot(
        data=df,
        x=f"param{sfx}",
        y=f"param_values{sfx}",
        hue="grp",
        hue_order=["unstruc", "struc"],
        ax=ax,
        palette=colors_2arm(),
    )
    ax.set_ylim(0, 11)
    fix_legend(ax)

In [ ]:
model.simulate_posterior_predictive()

In [ ]:
import seaborn as sns
from neuropy import plotting
from statplotannot.plots import xtick_format

fig = plotting.Fig(4, 2, fontsize=12)

ax = fig.subplot(fig.gs[0, 0])

sns.barplot(
    data=ucb_df,
    x="param_all",
    y="param_values_all",
    errorbar="se",
    hue="grp",
    ax=ax,
)
xtick_format(ax, rotation=45)